Link: https://www.kaggle.com/datasets/datafiniti/fast-food-restaurants?select=Datafiniti_Fast_Food_Restaurants_May19.csv

In [1]:
import re
import pandas as pd

import difflib

In [3]:
df = pd.read_csv('./FastFood.csv')
df

,id,dateAdded,dateUpdated,address,categories,primaryCategories,city,country,keys,latitude,longitude,name,postalCode,province,sourceURLs,websites
0,AWrSh_KgsVYjT2BJAzaH,2019-05-19T23:58:05Z,2019-05-19T23:58:05Z,2555 11th Avenue,"Fast Food Restaurants,Hamburgers and Hot Dogs,...",Accommodation & Food Services,Greeley,US,us/co/greeley/255511thavenue/554191587,40.396290,-104.696990,Carl's Jr.,80631,CO,https://www.yellowpages.com/greeley-co/mip/car...,https://www.carlsjr.com/?utm_source=Yextandutm...
1,AWEKlA-LIxWefVJwxG9B,2018-01-18T18:30:23Z,2019-05-19T23:45:05Z,2513 Highway 6 And 50,"Restaurant,Mexican Restaurants,Fast Food Resta...",Accommodation & Food Services,Grand Junction,US,us/co/grandjunction/2513highway6and50/1550891556,39.081350,-108.586890,Del Taco,81505,CO,http://www.citysearch.com/profile/772076870/gr...,http://www.deltaco.com
2,AWrSfAcYsVYjT2BJAzPt,2019-05-19T23:45:04Z,2019-05-19T23:45:04Z,1125 Patterson Road,"Sandwich Shops,Fast Food Restaurants,Restauran...",Accommodation & Food Services,Grand Junction,US,us/co/grandjunction/1125pattersonroad/-2137447852,39.091480,-108.554110,Which Wich,81506,CO,https://www.yellowpages.com/grand-junction-co/...,http://www.whichwich.com
3,AWrSa3NAQTFama1Xpkbz,2019-05-19T23:26:58Z,2019-05-19T23:26:58Z,3455 N Salida Court,"Fast Food Restaurants,Mexican Restaurants,Rest...",Accommodation & Food Services,Aurora,US,us/co/aurora/3455nsalidacourt/1143321601,39.763690,-104.776710,Chipotle Mexican Grill,80011,CO,https://www.yellowpages.com/aurora-co/mip/chip...,http://www.chipotle.com
4,AWrSaVGzZ4Yw-wtdgcaB,2019-05-19T23:24:38Z,2019-05-19T23:24:38Z,5225 E Colfax Avenue,"Fast Food Restaurants,Mexican Restaurants,Rest...",Accommodation & Food Services,Denver,US,us/co/denver/5225ecolfaxavenue/-864103396,39.740440,-104.926360,Taco Bell,80220,CO,https://www.yellowpages.com/denver-co/mip/taco...,https://locations.tacobell.com/co/denver/5225-...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,AVwd7U0T_7pvs4fz-fS_,2016-05-31T13:33:45Z,2018-12-03T09:04:19Z,1803 Carolina Ave,"American Restaurants,Caterers,Soul Food Restau...",Accommodation & Food Services,Bessemer,US,us/al/bessemer/1803carolinaave/785498836,33.399574,-86.951050,Church's Chicken Of Birmingham,35020,AL,http://www.yellowpages.com/bessemer-al/mip/chu...,http://www.churchs.com
9996,AV1XW1fF3D1zeR_xFKHL,2017-07-18T20:19:50Z,2018-12-02T00:08:36Z,8511 Cantilever Way Ste 105,"Mexican Restaurants,Caterers,Delicatessens,Iri...",Accommodation & Food Services,Raleigh,US,us/nc/raleigh/8511cantileverwayste105/-1596320866,35.884500,-78.737541,Luxury Box,27613-6996,NC,https://www.bbb.org/us/nc/raleigh/profile/spor...,"http://www.theluxuryboxofraleigh.com,http://th..."
9997,AV5UVbkiRxPSIh2RiQxV,2017-09-05T23:05:50Z,2018-12-01T07:38:03Z,2601 Plover Rd,"Carry-out food,Hot dog restaurants,Fast Food R...",Accommodation & Food Services,Plover,US,us/wi/plover/2601ploverrd/-1161002137,44.453180,-89.543170,McDonald's,54467,WI,https://www.yellowpages.com/plover-wi/mip/mcdo...,"http://mcdonalds.com,http://www.mcdonalds.com/..."
9998,AVwc9BxYByjofQCxj_GW,2016-03-31T02:43:12Z,2018-12-01T07:35:22Z,170 E Main St,"Carry-out food,Caterers,Restaurant delivery se...",Accommodation & Food Services,Hendersonville,US,us/tn/hendersonville/170emainst/-66712705,36.308262,-86.607339,Pizza Hut,37075,TN,https://foursquare.com/v/pizza-hut/4c5fffd43a3...,http://www.pizzahut.com


In [4]:
columnsToBeDropped = ['dateAdded','dateUpdated', 'primaryCategories','country','keys', 'latitude', 'longitude', 'postalCode', 'sourceURLs']
df.drop(columns = columnsToBeDropped, inplace=True)
df.head()

,id,address,categories,city,name,province,websites
0,AWrSh_KgsVYjT2BJAzaH,2555 11th Avenue,"Fast Food Restaurants,Hamburgers and Hot Dogs,...",Greeley,Carl's Jr.,CO,https://www.carlsjr.com/?utm_source=Yextandutm...
1,AWEKlA-LIxWefVJwxG9B,2513 Highway 6 And 50,"Restaurant,Mexican Restaurants,Fast Food Resta...",Grand Junction,Del Taco,CO,http://www.deltaco.com
2,AWrSfAcYsVYjT2BJAzPt,1125 Patterson Road,"Sandwich Shops,Fast Food Restaurants,Restauran...",Grand Junction,Which Wich,CO,http://www.whichwich.com
3,AWrSa3NAQTFama1Xpkbz,3455 N Salida Court,"Fast Food Restaurants,Mexican Restaurants,Rest...",Aurora,Chipotle Mexican Grill,CO,http://www.chipotle.com
4,AWrSaVGzZ4Yw-wtdgcaB,5225 E Colfax Avenue,"Fast Food Restaurants,Mexican Restaurants,Rest...",Denver,Taco Bell,CO,https://locations.tacobell.com/co/denver/5225-...


In [5]:
REPLACE_BY_SPACE_RE = re.compile('/|-|:|\.')
def createLowerCaseCityDictionary(df):
    lowerToUpperCity = {}
    for n in df['city']:
        word = n.lower().replace("'", "")
        word = REPLACE_BY_SPACE_RE.sub(' ', word)
        word = word.strip()
        lowerToUpperCity[word] = n
    return lowerToUpperCity

createLowerCaseCityDictionary(df)

In [6]:
def createLowerCaseNameDictionary(df):
    lowerToUpperName = {}
    lowerNameList = []
    for n in df['name']:
        
        word = n.lower().replace("'", "")
        word = REPLACE_BY_SPACE_RE.sub(' ', word)
        word = word.strip()
        word = re.findall(r'\w+', word)
        word = ' '.join(word)
        
        
        lowerToUpperName[word] = n
        lowerNameList.append(word)
    return lowerToUpperName, lowerNameList

lowerToUpperName, lowerNameList = createLowerCaseNameDictionary(df)
df['lowerNames'] = lowerNameList
df.head()

,id,address,categories,city,name,province,websites,lowerNames
0,AWrSh_KgsVYjT2BJAzaH,2555 11th Avenue,"Fast Food Restaurants,Hamburgers and Hot Dogs,...",Greeley,Carl's Jr.,CO,https://www.carlsjr.com/?utm_source=Yextandutm...,carls jr
1,AWEKlA-LIxWefVJwxG9B,2513 Highway 6 And 50,"Restaurant,Mexican Restaurants,Fast Food Resta...",Grand Junction,Del Taco,CO,http://www.deltaco.com,del taco
2,AWrSfAcYsVYjT2BJAzPt,1125 Patterson Road,"Sandwich Shops,Fast Food Restaurants,Restauran...",Grand Junction,Which Wich,CO,http://www.whichwich.com,which wich
3,AWrSa3NAQTFama1Xpkbz,3455 N Salida Court,"Fast Food Restaurants,Mexican Restaurants,Rest...",Aurora,Chipotle Mexican Grill,CO,http://www.chipotle.com,chipotle mexican grill
4,AWrSaVGzZ4Yw-wtdgcaB,5225 E Colfax Avenue,"Fast Food Restaurants,Mexican Restaurants,Rest...",Denver,Taco Bell,CO,https://locations.tacobell.com/co/denver/5225-...,taco bell


In [7]:
def correctName(spelledName, lowerToUpperName):
    lowerToUpperNameList = set(list(lowerToUpperName.keys()))
    spelledName.lower()
    
    closestNames = difflib.get_close_matches(spelledName, lowerToUpperNameList, n=6)
    for cn in closestNames:
        print(lowerToUpperName[cn])
    
    return closestNames

In [8]:
correctName('mcdonalss', lowerToUpperName)

McDonald's
Mc Donald's
McDonald's - CLOSED


['mcdonalds', 'mc donalds', 'mcdonalds closed']

In [ ]:
df['lowerCity'] = df['city'].str.lower()

In [9]:
def cleanCategories(s):
    extra_words = ['fast','foods', 'food', 'restaurants', 'restaurant', 'shops', 'shop', 'places', 'place']
    
    s = s.lower()
    for ex in extra_words:
        s = s.replace(ex,' ')
    s = re.split(',| and |&|/', s.strip())
    
    cleanWords = []
    for w in s:
        w = re.findall(r'\w+', w)
        w = ' '.join(w)
        if w != '': cleanWords.append(w)
        
    return list(set(cleanWords))

s = 'Pizza,Beer/Wine,Italian,Take-out,American,Lacto,Fast food,Take Out Restaurants,Vegan-friendly,Non-veg,Restaurants, Lacto'
cleanCategories(s)

['take out',
 'lacto',
 'american',
 'beer',
 'wine',
 'italian',
 'vegan friendly',
 'non veg',
 'pizza']

In [10]:
df['cleanCategories'] = df['categories'].apply(cleanCategories)
df

,id,address,categories,city,name,province,websites,lowerNames,cleanCategories
0,AWrSh_KgsVYjT2BJAzaH,2555 11th Avenue,"Fast Food Restaurants,Hamburgers and Hot Dogs,...",Greeley,Carl's Jr.,CO,https://www.carlsjr.com/?utm_source=Yextandutm...,carls jr,"[hot dogs, hamburgers]"
1,AWEKlA-LIxWefVJwxG9B,2513 Highway 6 And 50,"Restaurant,Mexican Restaurants,Fast Food Resta...",Grand Junction,Del Taco,CO,http://www.deltaco.com,del taco,[mexican]
2,AWrSfAcYsVYjT2BJAzPt,1125 Patterson Road,"Sandwich Shops,Fast Food Restaurants,Restauran...",Grand Junction,Which Wich,CO,http://www.whichwich.com,which wich,"[sandwich, take out]"
3,AWrSa3NAQTFama1Xpkbz,3455 N Salida Court,"Fast Food Restaurants,Mexican Restaurants,Rest...",Aurora,Chipotle Mexican Grill,CO,http://www.chipotle.com,chipotle mexican grill,"[take out, mexican]"
4,AWrSaVGzZ4Yw-wtdgcaB,5225 E Colfax Avenue,"Fast Food Restaurants,Mexican Restaurants,Rest...",Denver,Taco Bell,CO,https://locations.tacobell.com/co/denver/5225-...,taco bell,[mexican]
...,...,...,...,...,...,...,...,...,...
9995,AVwd7U0T_7pvs4fz-fS_,1803 Carolina Ave,"American Restaurants,Caterers,Soul Food Restau...",Bessemer,Church's Chicken Of Birmingham,AL,http://www.churchs.com,churchs chicken of birmingham,"[break brunch, chicken, caterers, american, so..."
9996,AV1XW1fF3D1zeR_xFKHL,8511 Cantilever Way Ste 105,"Mexican Restaurants,Caterers,Delicatessens,Iri...",Raleigh,Luxury Box,NC,"http://www.theluxuryboxofraleigh.com,http://th...",luxury box,"[hamburgers, sports bar, sandwich, caterers, i..."
9997,AV5UVbkiRxPSIh2RiQxV,2601 Plover Rd,"Carry-out food,Hot dog restaurants,Fast Food R...",Plover,McDonald's,WI,"http://mcdonalds.com,http://www.mcdonalds.com/...",mcdonalds,"[hamburgers, carry out, hamburger, american, g..."
9998,AVwc9BxYByjofQCxj_GW,170 E Main St,"Carry-out food,Caterers,Restaurant delivery se...",Hendersonville,Pizza Hut,TN,http://www.pizzahut.com,pizza hut,"[delivery service, carry out, caterers, americ..."


In [ ]:
# df.to_csv('FastFoodCleaned.csv')

In [ ]:
with open('restaurants.txt', 'w', encoding="utf-8") as f:
    cities = set(df['city'])
    for c in cities:
        f.write('- '+c.lower())
        f.write('\n')

In [ ]:
with open('cuisines.txt', 'w', encoding="utf-8") as f:
    categories = set(df['cleanCategories'])
    setCategories = set()
    for row in categories:
        for c in eval(row):
            setCategories.add(c)
    for c in setCategories:
        f.write('- '+c.lower())
        f.write('\n')